In [1]:
import time
import sys
sys.path.append('../')

import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from helpers.pytorch_helper import *
from helpers.data_helper import *

In [2]:
train = InstacartDataset('training.npy','training_labels.npy', transform=ToTensor())
validation = InstacartDataset('validation.npy','validation_labels.npy', transform=ToTensor())

In [3]:
BATCH = 32
train_loader = DataLoader(train, batch_size=BATCH, 
                          shuffle=True, num_workers=10)
val_loader = DataLoader(validation, batch_size=BATCH,
                        shuffle=True, num_workers=10)

In [4]:
class TesterLSTM(nn.Module):
    def __init__(self):
        super(TesterLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=148, 
                            hidden_size=512,
                            num_layers=3,
                            batch_first=False,
                            bidirectional=False)
        self.fc = nn.Linear(512*67,145)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (Variable(torch.zeros(3, 67, 512)).cuda(), # shape is (num_layers,sequence_length,hidden_dim)
                Variable(torch.zeros(3, 67, 512)).cuda())
    
    def forward(self, x):
        hidden = self.init_hidden()
        output, hidden = self.lstm(x, hidden)
        output = self.fc(output.view(x.size()[0],-1))
        return torch.clamp(output,0,49688)

In [5]:
tester = TesterLSTM().cuda()

In [6]:
LR = .01
EPOCHS = 200
loss_function = nn.MSELoss()
optimizer = torch.optim.RMSprop(tester.parameters(), lr=LR)

In [ ]:
best_val_loss = float('inf')
for epoch in range(EPOCHS):  # loop over the dataset multiple times
    epoch_loss = 0.
    running_loss = 0.
    time0 = time.time()
    # Training
    tester.train()
    for i, batch in enumerate(train_loader):
        # get the inputs
        inputs, labels = batch['features'], batch['target']
        # wrap them in Variable
        inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = tester(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.data[0]
        epoch_loss += loss.data[0]
        if i % 1000 == 0:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / (i+1)))
            running_loss = 0.0
    print('Epoch %d finished. Loss: %.5f' % (epoch+1,epoch_loss/(i+1)))
    print('Epoch %d took %.3f seconds.' % (epoch+1, time.time()-time0))
    
    # Validation
    val_loss = 0.
    tester.eval()
    for i, batch in enumerate(val_loader):
        inputs, labels = batch['features'], batch['target']
        inputs, labels = Variable(inputs, volatile=True).cuda(), Variable(labels, volatile=True).cuda()
        outputs = tester(inputs)
        val_loss += loss_function(outputs, labels).data[0]
    # Save checkpoint
    is_best = val_loss < best_val_loss
    best_val_loss = min(val_loss, best_val_loss)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': tester.state_dict(),
        'best_val_loss': best_val_loss,
        'optimizer' : optimizer.state_dict(),
    }, is_best)

    print("Validation loss for epoch %d: %.5f" % (epoch+1, val_loss/(i+1)))
print('Finished Training')

In [7]:
print("=> loading checkpoint '{}'".format("model_best.pth.tar"))
checkpoint = torch.load("model_best.pth.tar")
start_epoch = checkpoint['epoch']
best_prec1 = checkpoint['best_val_loss']
tester.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
print("=> loaded checkpoint '{}' (epoch {})".format("model_best.pth.tar", checkpoint['epoch']))

=> loading checkpoint 'model_best.pth.tar'
=> loaded checkpoint 'model_best.pth.tar' (epoch 25)


In [8]:
F_train=0.
acc_train=0.
for i, batch in enumerate(train_loader):
    inputs, labels = batch['features'], batch['target']
    inputs, labels = Variable(inputs, volatile=True).cuda(), Variable(labels, volatile=True).cuda()
    outputs = tester(inputs)
    F_train_batch, acc_train_batch = classify_and_score(outputs, labels)
    F_train += F_train_batch
    acc_train += acc_train_batch
print("Training F: %0.4f.  Training Accuracy: %0.4f.".format(F_train/(i+1),acc_train/(i+1)))
F_val=0.
acc_val=0.
for i, batch in enumerate(val_loader):
    inputs, labels = batch['features'], batch['target']
    inputs, labels = Variable(inputs, volatile=True).cuda(), Variable(labels, volatile=True).cuda()
    outputs = tester(inputs)
    F_val_batch, acc_val_batch = classify_and_score(outputs, labels)
    F_val += F_val_batch
    acc_val += acc_val_batch
print("Validation F: %0.4f.  Validation Accuracy: %0.4f.".format(F_val/(i+1),acc_val/(i+1)))

ValueError: multiclass-multioutput is not supported